In [18]:
import json
import re
import bz2
import time

import regex_dir.regex_expressions as re_exp

WIKI_FILE_PATH_1 = "src/enwiki-latest-pages-articles-multistream1.xml-p1p41242.bz2"
WIKI_FILE_PATH_2 = "src/enwiki-latest-pages-articles-multistream10.xml-p4045403p5399366.bz2"
WIKI_FILE_PATH_3 = "src/enwiki-20220920-pages-meta-current.xml.bz2"
START_TIME = ""
LIST_OF_BOOKS_v1 = []
LIST_OF_BOOKS_v1_FOUND = []

In [3]:
class WikiParserRegex_v1:

    def __init__(self):
        print("Parsing started!")

    def create_record(self):
        LIST_OF_BOOKS_v1.append({
            'name': [],
            'author': [],
            'country': [],
            'language': [],
            'series': [],
            'genre': [],
            'pub_date': [],
            'pages': []
        })

    def append_item_detail(self, line):
        if '| name' in line:
            item = re_exp.regex_name(line)
            if item:
                LIST_OF_BOOKS_v1[-1]['name'].append(item)
        elif '| author' in line:
            item = re_exp.regex_other(str(line))
            if item:
                LIST_OF_BOOKS_v1[-1]['author'].append(item)
        elif '| country' in line:
            item = re_exp.regex_plain(str(line))
            if item:
                LIST_OF_BOOKS_v1[-1]['country'].append(item)
        elif '| language' in line:
            item = re_exp.regex_plain(str(line))
            if item:
                LIST_OF_BOOKS_v1[-1]['language'].append(item)
        elif '| series' in line:
            item = re_exp.regex_other(str(line))
            if item:
                LIST_OF_BOOKS_v1[-1]['series'].append(item)
        elif '| genre' in line:
            item = re_exp.regex_other(str(line))
            if item:
                LIST_OF_BOOKS_v1[-1]['genre'].append(item)
        elif '| pages' in line:
            item = re_exp.regex_pages(str(line))
            if item:
                LIST_OF_BOOKS_v1[-1]['pages'].append(item)
        elif '| pub_date' in line or '| published' in line or '| release_date' in line or '| publish' in line:
            item = re_exp.regex_pub(str(line))
            if item:
                if not len(LIST_OF_BOOKS_v1[-1]['pub_date']) == 4:
                    LIST_OF_BOOKS_v1[-1]['pub_date'].append(item)

    def open_bz_file(self, path_to_file):
        counter = 0
        with bz2.BZ2File(path_to_file, "r") as xml_file:
            book_found = False
            created_record = False
            for idx, line in enumerate(xml_file):
                if line and "Infobox book" in str(line):
                    book_found = True
                    self.create_record()
                    created_record = True
                elif "}}" in str(line) and book_found:
                    book_found = False
                    # uncomment this if you want only 2 items from wiki as an example
                    #if counter == 5:
                    #    break
                    #counter = counter + 1
                elif book_found and created_record and line:
                    self.append_item_detail(str(line))

In [19]:
LIST_OF_BOOKS_v1 = []
start = time.time()
test = WikiParserRegex_v1()
test.open_bz_file(WIKI_FILE_PATH_3)
end = time.time()
print(end - start)

Parsing started!
7406.176096916199


In [20]:
#LIST_OF_BOOKS_v1 = []
print(len(LIST_OF_BOOKS_v1))

45842


In [56]:
for book in LIST_OF_BOOKS_v1:
    print(book)

In [21]:
"""
    If the category contains our input then return True

    @:param category_name is name of category (String)
    @:param category_input is the input we are looking for (String)
    @:param book is the book in which we are finding input (book from LIST_OF_BOOKS_v1)
"""
def find_my_category(category_name: str, category_input: str, book: any) -> bool:
    if category_input != '-' and category_name in book:
        for item in book[category_name]:
            for part in item:
                for part_find in category_input.split(', '):
                    if (category_name != "pub_date" or category_name != "pages") and part_find.lower() in part.lower():
                        return True
                    elif (category_name == "pub_date" or category_name == "pages") and part_find in part:
                        if int(part_find) > int(part):
                            return True
                        else:
                            return False
    elif category_input == '-':
        return True
    else:
        return False

In [22]:
def find_my_books(name: str, author: str, country: str, language: str, series: str, genre: str, pub_date: str, pages: str):
    LIST_OF_BOOKS_v1_FOUND.clear()

    #print(len(LIST_OF_BOOKS_v1))

    for book in LIST_OF_BOOKS_v1:
        name_bool = False
        author_bool = False
        country_bool = False
        language_bool = False
        series_bool = False
        genre_bool = False
        pub_bool = False
        pages_bool = False

        name_bool = find_my_category('name', name, book)
        author_bool = find_my_category('author', author, book)
        country_bool = find_my_category('country', country, book)
        language_bool = find_my_category('language', language, book)
        series_bool = find_my_category('series', series, book)
        genre_bool = find_my_category('genre', genre, book)
        pub_bool = find_my_category('pub_date', pub_date, book)
        pages_bool = find_my_category('pages', pages, book)

        if name_bool and author_bool and country_bool and language_bool and series_bool and genre_bool and pub_bool and pages_bool:
            LIST_OF_BOOKS_v1_FOUND.append(book)

In [23]:
def print_book_names():
    display_list = []
    for book in LIST_OF_BOOKS_v1_FOUND:
        if book['name'] not in display_list:
            display_list.append(book['name'])

    for item in display_list:
        if item:
            print(item[0])

In [24]:
name, author, country, language, series, genre, pub_date, pages = input("Hello in WikiParser!\n\n"
            "I've created an app that will search for the perfect set of books for you based on input parameters from the console!\n\n"
            "So LET'S GOO!!!\n\n"
            "Please insert the input from the keyboard, following these rules:\n"
            "- To separate multiple entries for a category use ',' and after this symbol use Space ' '\n"
            "- To separate categories use ';' and after this symbol use Space ' '\n"
            "- To skip a category, type '-'\n"
            "- !!! It is NECESSARY to enter all categories !!!\n\n"
            "The order of the categories is:\n"
            "- 'name'\n"
            "- 'author'\n"
            "- 'country'\n"
            "- 'language'\n"
            "- 'series'\n"
            "- 'genre'\n"
            "- 'pub_date'\n"
            "- 'pages'\n\n"
            "Thank you for using WikiParser!").split('; ')

"""
print(name.lower())
print(author.lower())
print(country.lower())
print(language.lower())
print(series.lower())
print(genre.lower())
print(pub_date.lower())
print(pages.lower())
.split(':')[0]
"""

LIST_OF_BOOKS_v1_FOUND.clear()

find_my_books(name, author, country, language, series, genre, pub_date, pages)

for book in LIST_OF_BOOKS_v1_FOUND:
    print(book)

print_book_names()

# -; J. K. Rowling; -; EN; -; Science, Fantasy; 2000; 300
# -; -; -; -; Harry Potter; -; -; -
# Harry Potter; J. K. Rowling; -; -; -; -; -; -
# Fantastic beasts; J. K. Rowling; -; -; -; -; -; -
# -; J. R. R. Tolkien; -; -; -; -; -; -
# A Tale of Two Cities; Charles Dickens; -; -; -; -; -; -
# -; Charles Dickens; -; -; -; -; -; -

{'name': [["Harry Potter and the Philosopher's Stone &lt;!-- The first edition was in the UK and was the Philosopher's Stone, NOT the Sorcerer's Stone. Read the second paragraph and do not change this! --&gt;"]], 'author': [['J. K. Rowling']], 'country': [['United Kingdom']], 'language': [['English']], 'series': [['Harry Potter']], 'genre': [], 'pub_date': [], 'pages': []}
{'name': [['Harry Potter and the Order of the Phoenix']], 'author': [['J. K. Rowling']], 'country': [['United Kingdom']], 'language': [['English']], 'series': [['Harry Potter']], 'genre': [], 'pub_date': [], 'pages': []}
{'name': [['Harry Potter and the Chamber of Secrets']], 'author': [['J. K. Rowling']], 'country': [['United Kingdom']], 'language': [['English']], 'series': [['Harry Potter']], 'genre': [['Fantasy novel']], 'pub_date': [], 'pages': []}
{'name': [['Harry Potter and the Prisoner of Azkaban']], 'author': [['J. K. Rowling']], 'country': [['United Kingdom']], 'language': [['English']], 'series': [['Harry 

In [73]:
item = 'u\xf1somestring'
decoded_value = item.encode('utf-8')
print(decoded_value)

b'u\xc3\xb1somestring'
